## Run cell2fate

In [ ]:
import anndata as ad
import cell2fate as c2f

from rgv_tools import DATA_DIR

## Import cell cycle dataset and train the model

In [ ]:
adata = ad.io.read_h5ad(DATA_DIR / "cell_cycle" / "cell_cycle_processed.h5ad")
adata

In [ ]:
clusters_to_remove = []
adata.obs["clusters"] = "0"

adata = c2f.utils.get_training_data(
    adata,
    cells_per_cluster=10**5,
    cluster_column="clusters",
    remove_clusters=clusters_to_remove,
    min_shared_counts=0,
    n_var_genes=2000,
)

In [ ]:
c2f.Cell2fate_DynamicalModel.setup_anndata(adata, spliced_label="spliced_raw", unspliced_label="unspliced_raw")

In [ ]:
n_modules = c2f.utils.get_max_modules(adata)

In [ ]:
mod = c2f.Cell2fate_DynamicalModel(adata, n_modules=n_modules)

In [ ]:
mod.train()

In [ ]:
adata = mod.export_posterior(
    adata, sample_kwargs={"batch_size": None, "num_samples": 30, "return_samples": True, "use_gpu": False}
)

In [ ]:
adata

In [ ]:
adata.write(DATA_DIR / "cell_cycle" / "cell2fate_filteredgene.h5ad")